In [1]:
# Install Required Packages if you don't have them:
# !pip install python-dotenv joblib anthropic hume-api

# Import Statements
import os
from dotenv import load_dotenv
from joblib import load
from pprint import pprint
import time
import traceback

# Custom Modules (Assuming these are in your project structure)
from src.utils.anthropicwrapper import ClaudeChat, ClaudeChatAssess
from src.utils.humewrapper import HumeSentimentAnalyzer
from src.modules import ConversationVerifier

import json
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

# --- Load Environment Variables ---
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))

# --- Timestamp Input ---
timestamp = 1725380720  # Example timestamp, replace with your input 

# --- Load Data ---
directory = f'data/interviews/{timestamp}/'
chatlog = load(os.path.join(directory, "joblib/conversation.joblib"))



In [3]:
# --- Model Setup --- 
try:
    sentiment_analyser = HumeSentimentAnalyzer(api_key=os.getenv("HUME_API_KEY"))

    sentiment_system_prompt = """You are a skilled emotions analyst provided with a detailed breakdown of sentiment analysis scores from Hume.ai, for a single response in an interview to a question from the interviewer. The scores are split into 3 sections. All numbers are from 0 to 1, linearly scaling, with 1 being a very strong representation of the indicator in question.

    First, Emotions. This contains several human emotions with a numerical value indicating the strength of the corresponding emotion.
    Second, Sentiments. This contains a scale from 1 to 9, each containing a numerical value indicating the magnitude of the sentiment of the topic of the conversation. A negative topic such as murder will have a high value lower in the scale, such as 1 or 2, and a positive topic will have a high value from 0 to 1 higher in the scale such as 8 or 9.
    Third, Toxicity. This contains several toxic representations such as hate, insult, etc, with a value from 0 to 1 for each representation identified in the audio.

    Your job is to provide a concise detailed one sentence breakdown of how the individual was feeling for the particular scores provided. You must be highly objective as your job is to discern whether or not a candidate was exhibiting traits which would or would not be fitting for a successful interview. 
    Model your answer beginning with something along the lines of "For this particular response, the candidate..."""
    
    sentiment_summariser = ClaudeChat("claude-3-5-sonnet-20240620", sentiment_system_prompt)

    evaluation_system_prompt = """You are a highly skilled interviewer currently tasked with reviewing a phone screening interview candidate to decide whether they are to pass on to the next stage of the interview process. There is a high volume of candidates and as such only 47% of candidates should be allowed to pass.
    You will be provided with the following information to aid your decision:
    1. A copy of the job description.
    2. Transcript of the phone interview between the interviewer and candidate.
    3. Sentiment analysis summary to provide you with insight into the tone of the candidate in the call
    4. A summary of any factual inaccuracies the candidate may have made during discussions in the call
    5. A copy of the candidate's CV, which will be provided next.

    You are to evaluate the candidate, primarily on the transcript, and use the additional information provided to identify any potential red-flags. Your response should include a detailed breakdown of why the candidate is chosen to continue onwards to further interviewing. You must end the breakdown with a simple one word response on a new line, "pass" or "fail"."""

    candidate_evaluator = ClaudeChatAssess("claude-3-5-sonnet-20240620", evaluation_system_prompt, "data/cvs/cv-deb.pdf")

except Exception as e:
    print(f"An error occurred during model setup: {e}")
    raise



Error processing PDF: no such file: 'data/cvs/cv-deb.pdf'


In [4]:
# --- Reformat Chatlog ---
def reformat_chatlog(chatlog):
    dropped_context = chatlog[3:]
    outputchatlog = []

    for i in range(0, len(dropped_context), 2):
        if i + 1 < len(dropped_context):
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': dropped_context[i+1]['content']
            }
            outputchatlog.append(tempdict)
        else:
            break 
            
    return outputchatlog

# --- Process Sentiments ---


In [5]:
def process_sentiments(chatlog_chat, timestamp):
    filepath = os.path.join('data', 'interviews', str(timestamp), 'audio')
    print(f"Current working directory: {os.getcwd()}")
    print(f"Full audio directory path: {os.path.abspath(filepath)}")

    files = [f for f in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, f))]
    
    if len(chatlog_chat) < len(files):
        files = files[:len(chatlog_chat)]

    for f in files:
        print(f"File found: {f}")

    sentiments = []
    
    for count, file in enumerate(files, 1):
        full_file_path = os.path.join(filepath, file)
        print(f"Processing file: {full_file_path}")
        
        # Add a small delay and re-check file existence
        time.sleep(0.1)
        if not os.path.exists(full_file_path):
            print(f"File not found (after delay): {full_file_path}")
            continue

        try:
            result = sentiment_analyser.analyze_audio(full_file_path)
            sentiment_summary = sentiment_summariser.chat(str(result))
            sentiments.append((result, sentiment_summary))
            chatlog_chat[count-1]['sentiment'] = sentiment_summary
        except Exception as e:
            print(f"Error processing file {full_file_path}: {str(e)}")
            traceback.print_exc()

    return chatlog_chat



In [6]:
# --- Evaluate Candidate --- 
def evaluate_candidate(chatlog_chat):
    ConversationVerifier.process_qa_pair(chatlog_chat)
    print("The Feedback JSON from the sentiment analyser and accuracy verifier: \n")
    pprint(chatlog_chat)
    evaluation = candidate_evaluator.chat(str(chatlog_chat))
    return evaluation



In [7]:
    # --- Main Execution Flow ---
for i in range(9):
        print(f'---------------------------{timestamp}-{i}------------------------')
        chatlog_chat = reformat_chatlog(chatlog)  # Process the loaded chatlog
        chatlog_chat = process_sentiments(chatlog_chat, timestamp) # Analyze sentiments

        #chatlog_chat_copy = chatlog_chat[1]
        evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation


        chatlog_file_path = f"data/interviews/{timestamp}-{i}/outcome/"

        if not os.path.exists(chatlog_file_path):
            os.makedirs(chatlog_file_path)

        with open(chatlog_file_path+"chatlog.json", "w") as file:
            json.dump(chatlog_chat, file, indent=4)
        evaluation_file_path = f"data/interviews/{timestamp}-{i}/outcome/"
        if not os.path.exists(chatlog_file_path):
            os.makedirs(chatlog_file_path)
        with open(evaluation_file_path+"evaluation.txt", "w") as file:
            file.write(str(evaluation).replace("\\n", "\n"))
        print(f'---------------------------{timestamp}-{i}------------------------')


---------------------------1725380262-0------------------------
Current working directory: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM
Full audio directory path: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\data\interviews\1725380262\audio
File found: audio_1_1725380262.wav
File found: audio_2_1725380262.wav
File found: audio_3_1725380262.wav
Processing file: data\interviews\1725380262\audio\audio_1_1725380262.wav
Analyzing audio file: data\interviews\1725380262\audio\audio_1_1725380262.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_1_1725380262.wav', 'content_type': 'audio/wav', 'md5sum': 'e4d8963ead06c2c7c810b7824635f21b'}, 'results': {'predictions': [], 'errors': [{'file': 'audio_1_1725380262.wav', 'model': None, 'message': 'unable to transcribe file: transcript confidence (0.0) below threshold value (0.3)'}]}}]
Error analyzing audio: list index out of range
Error processing file data\interv

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_22796\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 93, in _process_predictions
    pred = predictions[0]['results']['predictions'][0]['models']['language']['grouped_predictions'][0]['predictions'][0]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Analyzing audio file: data\interviews\1725380262\audio\audio_2_1725380262.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_2_1725380262.wav', 'content_type': 'audio/wav', 'md5sum': '114d1abf6761feb3828d34b8428abc49'}, 'results': {'predictions': [{'file': 'audio_2_1725380262.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.94677734, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "I'm sure. I currently work as a part and this receptionist to the university of Kent, and I'm looking to transition into... I mean, I'm looking to move to Her edinburgh, and this would be a good fit for me. Since I to t have the experience", 'position': {'begin': 0, 'end': 239}, 'time': {'begin': 0.35999998, 'end': 18.419044}, 'confidence': None, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.018388506025075912}, {'name': 'Adoration', 'score': 

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_22796\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 93, in _process_predictions
    pred = predictions[0]['results']['predictions'][0]['models']['language']['grouped_predictions'][0]['predictions'][0]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Analyzing audio file: data\interviews\1725380262\audio\audio_2_1725380262.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_2_1725380262.wav', 'content_type': 'audio/wav', 'md5sum': '114d1abf6761feb3828d34b8428abc49'}, 'results': {'predictions': [{'file': 'audio_2_1725380262.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.9472656, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "I'm sure. I currently work as a part and this receptionist to the university of Kent, and I'm looking to transition into... I mean, I'm looking to move to Her edinburgh, and this would be a good fit for me. Since I to t have the experience", 'position': {'begin': 0, 'end': 239}, 'time': {'begin': 0.35999998, 'end': 18.419044}, 'confidence': None, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.018388506025075912}, {'name': 'Adoration', 'score': 0

...

Analyzing audio file: data\interviews\1725380262\audio\audio_2_1725380262.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_2_1725380262.wav', 'content_type': 'audio/wav', 'md5sum': '114d1abf6761feb3828d34b8428abc49'}, 'results': {'predictions': [{'file': 'audio_2_1725380262.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.9472656, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "I'm sure. I currently work as a part and this receptionist to the university of Kent, and I'm looking to transition into... I mean, I'm looking to move to Her edinburgh, and this would be a good fit for me. Since I to t have the experience", 'position': {'begin': 0, 'end': 239}, 'time': {'begin': 0.35999998, 'end': 18.419044}, 'confidence': None, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.018388506025075912}, {'name': 'Adoration', 'score': 0

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_22796\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 93, in _process_predictions
    pred = predictions[0]['results']['predictions'][0]['models']['language']['grouped_predictions'][0]['predictions'][0]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Analyzing audio file: data\interviews\1725380262\audio\audio_2_1725380262.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_2_1725380262.wav', 'content_type': 'audio/wav', 'md5sum': '114d1abf6761feb3828d34b8428abc49'}, 'results': {'predictions': [{'file': 'audio_2_1725380262.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.9477539, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "I'm sure. I currently work as a part and this receptionist to the university of Kent, and I'm looking to transition into... I mean, I'm looking to move to Her edinburgh, and this would be a good fit for me. Since I to t have the experience", 'position': {'begin': 0, 'end': 239}, 'time': {'begin': 0.35999998, 'end': 18.419044}, 'confidence': None, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.018388506025075912}, {'name': 'Adoration', 'score': 0

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_22796\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 93, in _process_predictions
    pred = predictions[0]['results']['predictions'][0]['models']['language']['grouped_predictions'][0]['predictions'][0]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Analyzing audio file: data\interviews\1725380262\audio\audio_2_1725380262.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_2_1725380262.wav', 'content_type': 'audio/wav', 'md5sum': '114d1abf6761feb3828d34b8428abc49'}, 'results': {'predictions': [{'file': 'audio_2_1725380262.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.9477539, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "I'm sure. I currently work as a part and this receptionist to the university of Kent, and I'm looking to transition into... I mean, I'm looking to move to Her edinburgh, and this would be a good fit for me. Since I to t have the experience", 'position': {'begin': 0, 'end': 239}, 'time': {'begin': 0.35999998, 'end': 18.419044}, 'confidence': None, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.018388506025075912}, {'name': 'Adoration', 'score': 0

In [9]:

print(f'---------------------------{timestamp}------------------------')
chatlog_chat = reformat_chatlog(chatlog)  # Process the loaded chatlog
chatlog_chat = process_sentiments(chatlog_chat, timestamp) # Analyze sentiments

#chatlog_chat_copy = chatlog_chat[1]
evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation


chatlog_file_path = f"data/interviews/{timestamp}/outcome/"

if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)

with open(chatlog_file_path+"chatlog.json", "w") as file:
    json.dump(chatlog_chat, file, indent=4)
evaluation_file_path = f"data/interviews/{timestamp}/outcome/"
if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)
with open(evaluation_file_path+"evaluation.txt", "w") as file:
    file.write(str(evaluation).replace("\\n", "\n"))
print(f'---------------------------{timestamp}------------------------')

---------------------------1725236280------------------------
Current working directory: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM
Full audio directory path: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\data\interviews\1725236280\audio
File found: audio_1_1725236280.wav
Processing file: data\interviews\1725236280\audio\audio_1_1725236280.wav


Analyzing audio file: data\interviews\1725236280\audio\audio_1_1725236280.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_1_1725236280.wav', 'content_type': 'audio/wav', 'md5sum': '64933daa432855048bca1d9b44268a8b'}, 'results': {'predictions': [{'file': 'audio_1_1725236280.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.79833984, 'detected_language': 'hi'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': 'Hello.', 'position': {'begin': 0, 'end': 6}, 'time': {'begin': 0.62, 'end': 0.74}, 'confidence': 0.99072266, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.01251170877367258}, {'name': 'Adoration', 'score': 0.008131410926580429}, {'name': 'Aesthetic Appreciation', 'score': 0.016512403264641762}, {'name': 'Amusement', 'score': 0.0907270684838295}, {'name': 'Anger', 'score': 0.0005581923178397119}, {'name': 'Annoyance', 'score': 0.014846810139715

KeyboardInterrupt: 

In [7]:
chatlog_chat = [
  {
    "interviewer": "Hello! Thank you for taking the time to speak with me today about the Entry Level Tech Reviewer role. I'd like to start by asking you a question related to current technology. What is the latest Pixel Fold model?",
    "candidate": " The latest Pixel Fold model is the Pixel 9 Pro Fold.",
    "sentiment": "For this particular response, the candidate exhibited a notable level of calmness (0.32) and interest (0.22), along with moderate levels of positive surprise (0.16) and boredom (0.11), suggesting a composed and engaged demeanor with occasional moments of unexpected intrigue, while also potentially indicating some periods of disengagement or lack of enthusiasm during the interview."
  }
]

In [8]:
timestamp = 1725236280
evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation
evaluation = candidate_evaluator.chat(str(chatlog_chat))
    
chatlog_file_path = f"data/interviews/{timestamp}/outcome/"

if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)

with open(chatlog_file_path+"chatlog.json", "w") as file:
    json.dump(chatlog_chat, file, indent=4)
evaluation_file_path = f"data/interviews/{timestamp}/outcome/"
if not os.path.exists(chatlog_file_path):
    os.makedirs(chatlog_file_path)
with open(evaluation_file_path+"evaluation.txt", "w") as file:
    file.write(str(evaluation).replace("\\n", "\n"))
print(f'---------------------------{timestamp}------------------------')


attempting to generate searc queries from answers
search queries generated for answer to question: Hello! Thank you for taking the time to speak with me today about the Entry Level Tech Reviewer role. I'd like to start by asking you a question related to current technology. What is the latest Pixel Fold model?

queries are: ['1. "Pixel 9 Pro Fold release date"', '2. "Pixel 9 Pro Fold specifications and features"']


No documents retrieved from webscrapping 

Feedback is being returned from accuracy verifier
The Feedback JSON from the sentiment analyser and accuracy verifier: 

[{'candidate': ' The latest Pixel Fold model is the Pixel 9 Pro Fold.',
  'feedback': 'The accuracy of the answer provided is 0%. \n'
              '\n'
              'The answer states that the latest Pixel Fold model is the '
              '"Pixel 9 Pro Fold," which is incorrect. The latest model in the '
              'Pixel Fold series is the "Google Pixel Fold." There is no model '
              'called the 